In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import astroinject

from astroinject.pipeline.apply_index import apply_pgsphere_index, apply_q3c_index, apply_btree_index
from astroinject.pipeline.injection import injection_procedure, create_table, parallel_insertion

from astroinject.utils import find_files_with_pattern
from astroinject.config import load_config

from astroinject.pipeline.map_tap_schema import map_table

import warnings
from logpool import control

# Creating config

In [ ]:
config = {
    'database': 
    {
        'dbname': '',
        'user': '',
        'password': '',
        'host': '',
        'port': 5432
    },
    'general': {'injection_processes': 6},
    'folder': '/home/astrodados3/splus/idr6_final',
    'pattern': '*.fits',
    'tablename': 'teste',
    'force_cast_correction': False,
    'id_col': 'id',
    'ra_col': 'ra',
    'dec_col': 'dec',
    'index_type': 'pgsphere',
    'additional_btree_index': ['field'],
    'rename_columns': {},
    'delete_columns': [],
    'patterns_to_replace': [],
    'mask_value': 'None'
}

# injecting files written in disk

In [ ]:
control.info("starting injection procedure")
control.info(f"config: \n{config}")

files = find_files_with_pattern(
    config["folder"],
    config["pattern"]
)

control.info(f"found {len(files)} files to inject")

parallel_insertion(files, config)

if "additional_btree_index" in config and config["additional_btree_index"]:
    control.info("creating additional B-Tree index")
    apply_btree_index(config)

if not "index_type" in config or config["index_type"] is None:
    control.info("no index type specified, skipping index creation")
elif config["index_type"] == "pgsphere":
    apply_pgsphere_index(config)
elif config["index_type"] == "q3c":
    apply_q3c_index(config)

# Injecting files on memory

In [21]:
import numpy as np 
from astropy.table import Table

In [25]:
# create table with random data using numpy

tab = Table()
tab['id'] = np.random.rand(100)
tab['ra'] = np.random.rand(100)
tab['dec'] = np.random.rand(100)

In [30]:
create_table(tab, config)

01/09/2025 11:04:08  [info] - 660484 - injection.py - create_table() - Creating table teste in the database
01/09/2025 11:04:08  [info] - 660484 - injection.py - create_table() - Query: 
CREATE TABLE IF NOT EXISTS teste (
    id FLOAT8 PRIMARY KEY,
    ra FLOAT8 NULL,
    dec FLOAT8 NULL
);


In [31]:
from astroinject.database.types import build_type_map
types_map = build_type_map(config) if config.get("force_cast_correction") else None

In [32]:
injection_procedure(tab, types_map, config)

01/09/2025 11:04:10  [info] - 660484 - injection.py - injection_procedure() - Injecting table Memory file. into the database
✅ Inserted 100 rows into teste using COPY (no conflict handling).
